In [51]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import os
# starts the spark only three steps
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [53]:
df = spark.read.csv('/content/drive/MyDrive/big data/week 4/bank (3).csv',inferSchema=True,header=True)

In [54]:
df.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|     admin.| married| tertiary|     no|    184|     no|  

In [55]:
print(df.count())

11162


In [56]:
df.select("deposit").distinct().show()
df.printSchema()


+-------+
|deposit|
+-------+
|     no|
|    yes|
+-------+

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



check classes are balanced or Not

In [57]:
print("yes",df.select("deposit").where("deposit=='yes'").count())
print("No",df.select("deposit").where("deposit=='no'").count())

yes 5289
No 5873


checking the distinct values in all string columns such as:
Jobs
Education
martial status
etc of the dataframe using functional programming

In [58]:
# checking the distinct values in string columns of the dataframe using functional programming 
[print("column name",df[t[0]].name,df.select(t[0]).distinct().show()) for t in df.dtypes if t[1]=='string' ]

+-------------+
|          job|
+-------------+
|   management|
|      retired|
|      unknown|
|self-employed|
|      student|
|  blue-collar|
| entrepreneur|
|       admin.|
|   technician|
|     services|
|    housemaid|
|   unemployed|
+-------------+

column name <bound method alias of Column<'job'>> None
+--------+
| marital|
+--------+
|divorced|
| married|
|  single|
+--------+

column name <bound method alias of Column<'marital'>> None
+---------+
|education|
+---------+
|  unknown|
| tertiary|
|secondary|
|  primary|
+---------+

column name <bound method alias of Column<'education'>> None
+-------+
|default|
+-------+
|     no|
|    yes|
+-------+

column name <bound method alias of Column<'default'>> None
+-------+
|housing|
+-------+
|     no|
|    yes|
+-------+

column name <bound method alias of Column<'housing'>> None
+----+
|loan|
+----+
|  no|
| yes|
+----+

column name <bound method alias of Column<'loan'>> None
+---------+
|  contact|
+---------+
|  unknown|
| cell

[None, None, None, None, None, None, None, None, None, None]

Displaying only integer columns


In [59]:
# taking only integer features
 
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,11162,41.231947679627304,11.913369192215518,18,95
balance,11162,1528.5385235620856,3225.413325946149,-6847,81204
day,11162,15.658036194230425,8.420739541006462,1,31
duration,11162,371.99381831213043,347.12838571630687,2,3881
campaign,11162,2.508421429851281,2.7220771816614824,1,63
pdays,11162,51.33040673714388,108.75828197197717,-1,854
previous,11162,0.8325568894463358,2.292007218670508,0,58


In [60]:
#find out is there any null value in any column
print("total records:",df.count())
[print(col,"Total null values",df.where(df[col].isNull()).count()) for col in df.columns]

total records: 11162
age Total null values 0
job Total null values 0
marital Total null values 0
education Total null values 0
default Total null values 0
balance Total null values 0
housing Total null values 0
loan Total null values 0
contact Total null values 0
day Total null values 0
month Total null values 0
duration Total null values 0
campaign Total null values 0
pdays Total null values 0
previous Total null values 0
poutcome Total null values 0
deposit Total null values 0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Selecting Feature for Feature Vector:
Input variables: age, job, marital, education, default, balance, housing, loan, contact, day, month, duration, campaign, pdays, previous, poutcome. Out PutVariable:variable: deposit , we will keep all of numeric variable for the model. However, day and month columns are not really useful, we will remove these two columns.

In [61]:
# Non Numeric features
ndf = df.select('job', 'marital', 'education', 'default',  'housing', 'loan', 'contact', 'duration',  'poutcome', 'deposit')
cols = ndf.columns
ndf.printSchema()

root
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



Taking both Numeric and String cols


Change Categorial Variable using one hot encoding
Step 1: apply string index Step 2: using String indexing to apply one hot Encoding Step 3: Create Vector Assembler

In [62]:
# Now we will apply string indexing to categorial variables using pipe line 
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    print(stringIndexer.getOutputCol())
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
print(label_stringIdx)
stages += [label_stringIdx]
numericCols = ['age','balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
print(assemblerInputs)
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

jobIndex
maritalIndex
educationIndex
defaultIndex
housingIndex
loanIndex
contactIndex
poutcomeIndex
StringIndexer_5ba10bf7c5f2
['jobclassVec', 'maritalclassVec', 'educationclassVec', 'defaultclassVec', 'housingclassVec', 'loanclassVec', 'contactclassVec', 'poutcomeclassVec', 'age', 'balance', 'duration', 'campaign', 'pdays', 'previous']


Now use this pipe line using fit and transform passing df as data

In [63]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(ndf)
ndf = pipelineModel.transform(df)
ndf.show(12,True)


+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+--------+--------------+------------+---------------+--------------+-----------------+------------+---------------+------------+---------------+---------+-------------+------------+---------------+-------------+----------------+-----+--------------------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|jobIndex|   jobclassVec|maritalIndex|maritalclassVec|educationIndex|educationclassVec|defaultIndex|defaultclassVec|housingIndex|housingclassVec|loanIndex| loanclassVec|contactIndex|contactclassVec|poutcomeIndex|poutcomeclassVec|label|            features|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+--------+--------------+------------+---------------+--------------+-----------------+---

In [64]:
# feature vector is in features  from assembler. 
selectedCols = ['label', 'features'] + cols
ndfSelected = ndf.select(selectedCols)
ndfSelected.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [65]:
# this is code for multiple classification using logistic Regression 
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LogisticRegression
train, test = ndfSelected.randomSplit([0.7, 0.3], seed = 2018)
lr = LogisticRegression(maxIter=100, \
                        
                        featuresCol="features", \
                        
                        labelCol='label')
ovr = OneVsRest(classifier=lr, \
                labelCol='label', \
                featuresCol='features')
#from pyspark.ml import Pipeline
#pipeline_ovr = Pipeline(stages=[vecAssembler, stdScaler, ovr])
#pipelineModel_ovr = pipeline_ovr.fit(trainDF)

ovrModel = ovr.fit(train)
predictionsovr = ovrModel.transform(test)
predictionsovr.show(10)

+-----+--------------------+----------+-------+---------+-------+-------+----+---------+--------+--------+-------+--------------------+----------+
|label|            features|       job|marital|education|default|housing|loan|  contact|duration|poutcome|deposit|       rawPrediction|prediction|
+-----+--------------------+----------+-------+---------+-------+-------+----+---------+--------+--------+-------+--------------------+----------+
|  0.0|(30,[0,11,13,16,1...|management|married|secondary|     no|     no|  no| cellular|      80| unknown|     no|[1.91889531585926...|       0.0|
|  0.0|(30,[0,11,13,16,1...|management|married|secondary|     no|     no|  no| cellular|      47| unknown|     no|[1.92010568650575...|       0.0|
|  0.0|(30,[0,11,13,16,1...|management|married|secondary|     no|     no|  no| cellular|     477| unknown|     no|[-0.7779866740363...|       1.0|
|  0.0|(30,[0,11,13,16,1...|management|married|secondary|     no|     no|  no| cellular|     140| unknown|     no|[0.9

Decision Tree Classifier

In [67]:
from pyspark.ml.classification import DecisionTreeClassifier
train, test = ndfSelected.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)

#predictions.select("label","prediction").show(10)
predictions.select( 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)
print("Total Actual Positive",predictions.select("label").where('label == 1.0').count())
print("Total Actual Negative",predictions.select("label").where('label == 0.0').count())
pr = predictions.toPandas()
TruePositive =0
FalsePositive=0
TrueNegative=0
FalseNegative=0
Postive=1.0
Negative=0.0
pos=0
Neg=0

print("Total",len(pr["label"]))
for lbl in range(len(pr["label"])):
  if  pr["prediction"][lbl]==Postive:
    pos+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TruePositive+=1
    else:
      FalsePositive+=1
  if  pr["prediction"][lbl]==Negative:
    Neg+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TrueNegative+=1
    else:
      FalseNegative+=1
print("Total Positive & Negative in Prediction. Pos: ",pos,",Neg",Neg)
print("TruePostive",TruePositive,"FalsePostive",FalsePositive)     
print("TrueNegative",TrueNegative,"FalseNegative",FalseNegative)    

Training Dataset Count: 7855
Test Dataset Count: 3307
+----------+-----+--------------+----------+--------------------+
|       job|label| rawPrediction|prediction|         probability|
+----------+-----+--------------+----------+--------------------+
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[520.0,1931.0]|       1.0|[0.21215830273357...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
+----------+-----+--------------+----------+--------------------+
only showing top 10 ro

AUC - ROC curve

In [69]:

#predictions.show(10)
#print(dtModel.toDebugString)
#Evaluate our Decision Tree model.

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()

print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.7808118726917547


In [72]:
#RandomForest
from pyspark.ml.classification import RandomForestClassifier
#rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rf = RandomForestClassifier(featuresCol='features', labelCol='label', maxDepth = 4,numTrees=20)
train, test = ndfSelected.randomSplit([0.7, 0.3], seed = 2018)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
pr = predictions.toPandas()
print("Total Actual Positive",predictions.select("label").where('label == 1.0').count())
print("Total Actual Negative",predictions.select("label").where('label == 0.0').count())

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
TruePositive =0
FalsePositive=0
TrueNegative=0
FalseNegative=0
Postive=1.0
Negative=0.0
pos=0
Neg=0
print("Total",len(pr["label"]))
for lbl in range(len(pr["label"])):
  if  pr["prediction"][lbl]==Postive:
    pos+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TruePositive+=1
    else:
      FalsePositive+=1
  if  pr["prediction"][lbl]==Negative:
    Neg+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TrueNegative+=1
    else:
      FalseNegative+=1
print("Total positive & Negative in Predictions, Pos",pos,"Neg",Neg)
print("TruePostive",TruePositive,"FalsePostive",FalsePositive)     
print("TrueNegative",TrueNegative,"FalseNegative",FalseNegative)     

Total Actual Positive 1603
Total Actual Negative 1704
Test Area Under ROC: 0.8660476322271329
Total 3307
Total positive & Negative in Predictions, Pos 1484 Neg 1823
TruePostive 1187 FalsePostive 297
TrueNegative 1407 FalseNegative 416


In [74]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

ml = GBTClassifier(maxIter=10, featuresCol='features', labelCol='label',maxDepth = 10)
train, test = ndfSelected.randomSplit([0.7, 0.2], seed = 2018)
mlModel = ml.fit(train)
predictions = mlModel.transform(test)

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

# Calculate accuracy and F-1 score
#accuracy_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
#accuracy = accuracy_evaluator.evaluate(predictions.select('label', 'prediction'))
    
#f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
#f1_score = f1_score_evaluator.evaluate(predictions.select('label', 'prediction'))
#print(accuracy, f1_score)

Test Area Under ROC: 0.869695789478623


In [75]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
gbt = GBTClassifier(maxIter=10)
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

0.8989346833243608

In [76]:
predictions.select('label', 'prediction').show(13)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 13 rows

